In [7]:
import autogen
from autogen.trace import trace_op, node
from autogen.trace.optimizers import FunctionOptimizer
from autogen.trace.nodes import GRAPH

GRAPH.clear()


def log(output, feedback, parameters):
    print(f"output={output}, feedback={feedback}, variables=\n")  # logging
    for p in parameters:
        print(p.name, p.data)


def user(output):
    return "Success." if output < 0 else "Try again. The output should be negative"


@trace_op(trainable=True)
def my_fun(x):
    """Test function"""
    return 0


x = node(-1, trainable=True)


def computation(x):
    y = my_fun(x)
    # additional computation here
    return y


optimizer = FunctionOptimizer(
    parameters=[x, my_fun.parameter], config_list=autogen.config_list_from_json("OAI_CONFIG_LIST")
)

while True:
    output = computation(x)
    feedback = user(output.data)
    if feedback == "Success.":
        break
    optimizer.zero_feedback()
    optimizer.backward(output, feedback)
    optimizer.step(verbose=False)

    log(output.data, feedback, optimizer.parameters)

output=0, feedback=Try again. The output should be negative, variables=

int:0 -1
__code:0 def my_fun(x):
    """Test function"""
    return x
